# L6d: Another Look at Solving Linear Equations
In this activity, we will revisit the solution of linear algebraic equations, where we will compare the solution that we obtain by using QR decomposition with our iterative methods for solving linear equations.

> **Learning objectives:**
>
> In this lab, students will learn to:
>
> * **Compare direct and iterative solution methods.** We solve a manufactured 1D Poisson equation using QR decomposition versus iterative methods (Jacobi, Gauss-Seidel, SOR) to understand their accuracy and convergence behavior.
> * **Analyze computational performance trade-offs.** We benchmark memory usage, execution time, and solution accuracy across different linear algebra approaches to determine optimal method selection.
> * **Validate numerical solutions using manufactured solutions.** We construct test problems with known exact solutions to measure and compare the precision of different linear equation solvers.

___
Let's get started!

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on Julia functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's setup our code environment:

In [1]:
include(joinpath(@__DIR__, "Include-solution.jl"));

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types and data used in this material. 

___

## Task 1: Setup the System of Linear Equations
In this task, we will set up a system of linear equations that we will solve using both QR decomposition and iterative methods. We will define the coefficient matrix $\mathbf{A}$ and the right-hand side vector $\mathbf{b}$ for the system of equations $\mathbf{A}\mathbf{x} = \mathbf{b}$.

> __Test Problem (method of manufactured solutions)__
> 
> Let's solve a classic problem: the **one-dimensional Poisson equation** with **Dirichlet boundary conditions**. This shows up everywhere in physics and engineering (heat conduction, electrostatics, fluid flow, you name it). However, instead of solving it directly, we'll use a trick called the **method of manufactured solutions** to create a test problem with a known solution. 
> 
> * __Idea:__ We pick $u(x) = \sin(\pi x)$ as our known answer, discretize the domain into $n$ grid points at $x_i = \frac{i}{n+1}$, and use finite differences to approximate derivatives. The continuous equation $-u''(x) = f(x)$ becomes the discrete system $-\frac{u_{i-1} - 2u_i + u_{i+1}}{h^2} = f_i$.
> * __Manufactured solution approach__: In the manufactured solution approach, we don't actually need to know what $f(x)$ is! Since we already decided that $u(x) = \sin(\pi x)$ is our solution, we can just compute $\mathbf{b} = \mathbf{A} \mathbf{x}_{true}$ directly. The vector $\mathbf{b}$ represents the discrete version of $f(x)$ that would produce our chosen solution. It's like saying "if this is the answer, what must the question have been?"
>
> This approach gives us a **tridiagonal system** $\mathbf{A}\mathbf{x} = \mathbf{b}$ where $\mathbf{A}$ has 2's on the diagonal and -1's on the off-diagonals. Since we know the exact answer, we can measure how accurate our different numerical methods actually are!

Let's setup our system of equations. We save the system matrix in the `A::Array{Float64,2}` variable, the right-hand side vector in the `b::Array{Float64,1}` variable and the true solution in the `x_true::Array{Float64,1}` variable.

In [2]:
A,b,x_true = let 
    
    n = 100; # how big of a system do we want to solve?

    # Build the tridiagonal matrix A and the manufactured solution x⋆ and right-hand side b
    # for the 1D Poisson equation with Dirichlet boundary conditions.
    # The matrix A is constructed using the sparse diagonal matrix constructor `spdiagm`.
    di = fill(2.0, n); dl = fill(-1.0, n-1); du = fill(-1.0, n-1)
    A  = diagm(-1 => dl, 0 => di, 1 => du)              # tridiagonal matrix A  
    xg = range(1, n; step=1) ./ (n+1)                   # grid i/(n+1)
    xstar = @. sin(pi * xg)                             # "true" solution
    b = A * xstar                                       # manufactured RHS

    A,b, xstar # return
end;

What does the system matrix $\mathbf{A}$ look like for this problem?

In [3]:
A

100×100 Matrix{Float64}:
  2.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   2.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   2.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   2.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   2.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  ⋮                             ⋮    ⋱         ⋮                      
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.

### Check: Is the system matrix $\mathbf{A}$ strictly diagonally dominant?
Before we continue, let's verify the generated system matrix $\mathbf{A}$ is actually diagonally dominant.
> __Test:__ We'll compute the sum of the absolute values of each row (excluding the diagonal element), and compare it to the absolute value of the diagonal element in that row. If the sum of the absolute values of the non-diagonal elements is less than the absolute value of the diagonal element, then the matrix is diagonally dominant.

Is the matrix $\mathbf{A}$ strictly diagonally dominant?

In [4]:
ddcondition = let
    
    # initialize -
    number_of_rows = size(A, 1);
    ddcondition = Array{Bool,1}(undef, number_of_rows);

    # let's check each row
    for i ∈ 1:number_of_rows
        aii = abs(A[i,i]);
        σ = 0.0;
        for j ∈ 1:number_of_rows
            if (i ≠ j)
                σ += abs(A[i,j]);
            end
        end
        ddcondition[i] = (aii ≥ σ) ? true : false; # ternary operator, nice! # TODO: Notice we use ≥ here, not >, so we allow for weak diagonal dominance
    end

    ddcondition
end;

The `ddcondition` array contains boolean values indicating whether each row of our system matrix $\mathbf{A}$ satisfies the diagonal dominance condition. Each element `ddcondition[i]` is `true` if the absolute value of the diagonal element in row `i` is greater than the sum of the absolute values of all other elements in that row, and `false` otherwise.

> __Test:__ The assertion `@assert any(ddcondition)` checks that at least one row (and hopefully all rows) satisfies the diagonal dominance condition. If the condition fails, a warning is issued indicating that the matrix may not be suitable for iterative methods, as convergence is not guaranteed.

For diagonally dominant matrices, iterative methods like Jacobi, Gauss-Seidel, and SOR are guaranteed to converge, making them reliable choices for solving linear systems.

In [5]:
try
    @assert any(ddcondition .== false) == false; # if any are false, then the assertion fails
catch
    @warn "Diagonal dominance condition not satisfied"
end

### Check: Is our test solution correct for the system of equations?
We generated a manufactured solution vector `x_true::Array{Float64,1}` and computed the corresponding right-hand side vector `b::Array{Float64,1}` using the system matrix `A::Array{Float64,2}`. Now, we will verify that our generated solution is indeed correct by checking if the equation $\mathbf{A} \mathbf{x}_{\text{true}} = \mathbf{b}$ holds true.

> __Test:__ We will compute the product of the system matrix `A` and the solution vector `x_true`, and compare it to the right-hand side vector `b`. If they are (approximately) equal, then our generated solution is correct. We will use [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check for approximate equality in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the test passes.

So what do we see?

In [6]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance for checking the residual norm against zero

    # compute the residual for the manufactured solution
    r = b - A*x_true # if r ≈ 0, then x_true is a solution to the system of equations Ax = b
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol);
end

If we get through without any warnings or assertion errors, then our generated solution is indeed correct for the system of equations defined by the matrix `A` and the vector `b`!
___

## Task 2: Compare the QR decomposition solution with the true solution
In this task, we will solve the system of linear equations using QR decomposition and compare the solution we obtain with the true solution vector `x_true` that we generated earlier.

> __QR Decomposition__
>
> The QR decomposition can be used as a __direct method__ for solving linear systems. The QR decomposition factors a matrix into two orthogonal matrices, $\mathbf{Q}$ and $\mathbf{R}$, such that $\mathbf{A} = \mathbf{Q} \mathbf{R}$. The matrix $\mathbf{Q}$ is orthogonal, meaning that its columns are orthonormal vectors, which gives us the property that $\mathbf{Q}^{\top} \mathbf{Q} = \mathbf{I}$, where $\mathbf{I}$ is the identity matrix. The matrix $\mathbf{R}$ is an upper triangular matrix.
>
> This is super handy for solving linear systems of equations:
>$$
>\begin{align*}
>\mathbf{A} \mathbf{x} &= \mathbf{b}\quad\Longrightarrow{\text{decompose}}\;\mathbf{A} = \mathbf{Q} \mathbf{R} \\
>\left(\mathbf{Q} \mathbf{R}\right) \mathbf{x} &= \mathbf{b}\quad\Longrightarrow\text{multiply by } \mathbf{Q}^{\top} \\
>\underbrace{\left(\mathbf{Q}^{\top} \mathbf{Q}\right)}_{=\mathbf{I}} \mathbf{R} \mathbf{x} &= \mathbf{Q}^{\top} \mathbf{b}\quad\Longrightarrow\text{solve for } \mathbf{x} \\
>\mathbf{x} &= \mathbf{R}^{-1} \mathbf{Q}^{\top} \mathbf{b}\quad\blacksquare
>\end{align*}
$$ 

QR decomposition is particularly useful for solving linear systems because it avoids the need to compute the inverse of the matrix $\mathbf{A}$ directly, which can be computationally expensive and numerically unstable. Let's compute the QR decomposition of the matrix `A`, solve for the solution vector `x_qr::Array{Float64,1}`.

In [7]:
x_qr = let

    Q,R = qr(A);
    x_qr = R \ (transpose(Q) * b); # TODO: Note that we use the backslash operator here to solve the triangular system Rx = Qᵀb

    x_qr # return the solution
end;

Next, compare `x_qr::Array{Float64,1}` and `x_true::Array{Float64,1}` using [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check for approximate equality.

> __Test:__ We will use the `isapprox(...)` function to check if the p = 2 norm of the difference between the two solution vectors is within a specified tolerance. If they are approximately equal, then our QR decomposition solution is correct. If not, an assertion error will be raised.

So what happens?

In [8]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance for checking the residual norm against zero

    # compute the norm of the difference between the computed solution and the manufactured solution
    r = x_qr - x_true; # if r ≈ 0, then x_qr is approximately equal to x_true
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol); # test
end

__Ok!__ If we didn't blow up, we are good. QR decomposition gives us a solution that is approximately equal to the true solution we generated earlier. This confirms that our QR decomposition approach is valid for solving the system of linear equations defined by the matrix `A` and the vector `b`.

How about our iterative methods? Do they also give us a solution that is approximately equal to the true solution `x_true`? Let's check that next!
___

## Task 3: Compare the iterative method solutions with the true solution
In this task, we will solve the system of linear equations using the iterative methods we have implemented (Jacobi, Gauss-Seidel, and SOR) and compare the solutions we obtain with the true solution vector `x_true` that we generated earlier.

Let's setup a code block that allows us to compute the solution using an iterative method of our choice, and then compare the solution to the true solution vector `x_true` using the `isapprox(...)` function to check for approximate equality.

In [9]:
iterative_solution_archive = let

    # initialize -
    number_of_rows = size(A, 1);
    maximum_number_of_iterations = 25; # maximum number of iterations for the iterative method
    tolerance = 1e-8;
    algorithm = GaussSeidelMethod(); # change this to Jacobi(), GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

    # call the solve method with the appropriate parameters -
    x = VLDataScienceMachineLearningPackage.solve(A, b, xₒ; algorithm = algorithm, 
        maxiterations = maximum_number_of_iterations, 
        ϵ = tolerance, 
        ω = ω # only used for SOR
    );

    x; # return the solution vector computed by the iterative method
end

┌ Warning: Gauss-Seidel method did not converge within 25 iterations. Final residual: 0.007085582532338561
└ @ VLDataScienceMachineLearningPackage /Users/jeffreyvarner/.julia/packages/VLDataScienceMachineLearningPackage/RymMA/src/Solvers.jl:93


Dict{Int64, Vector{Float64}} with 27 entries:
  5  => [0.0247083, 0.0453315, 0.0615716, 0.0737911, 0.0826585, 0.0889105, 0.09…
  16 => [0.0143657, 0.0279296, 0.0404132, 0.0516396, 0.0615262, 0.0700701, 0.07…
  20 => [0.0130112, 0.0254474, 0.0370878, 0.0477754, 0.0574163, 0.0659738, 0.07…
  12 => [0.016387, 0.0315545, 0.0451441, 0.0569749, 0.067016, 0.0753493, 0.0821…
  24 => [0.0120353, 0.0236348, 0.0346186, 0.04485, 0.0542368, 0.0627288, 0.0703…
  8  => [0.0198081, 0.0374528, 0.0524932, 0.0648461, 0.074677, 0.0822948, 0.088…
  17 => [0.0139798, 0.0272266, 0.0394781, 0.0505622, 0.0603911, 0.0689504, 0.07…
  1  => [0.050015, 0.0750376, 0.0875639, 0.093842, 0.0969959, 0.0985877, 0.0993…
  19 => [0.0133067, 0.0259923, 0.0378235, 0.048638, 0.0583429, 0.0669074, 0.074…
  0  => [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1  …  0.1, 0.1, 0.1, 0.…
  22 => [0.0124867, 0.0244756, 0.0357681, 0.0462176, 0.0557302, 0.0642608, 0.07…
  6  => [0.0226808, 0.0421563, 0.0580187, 0.0703987, 0.0797295,

### Check: Is the iterative solution actually a valid solution for the system of equations?
In this check, we will compute the residual vector `r` for the iterative solution obtained from our iterative method and compare its norm to zero. The residual vector is defined as `r = b - A * x_iterative`, where `x_iterative` is the solution obtained from the iterative method. 

> __Test:__ If the norm of the residual vector is approximately zero, then the iterative solution is a valid solution for the system of equations. Let's setup a code block that computes the residual vector and checks if its norm is approximately zero using [the `isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the test passes.

So what happens?

In [10]:
let

    # initialize -
    atol = 1e-10; # absolute tolerance
    i = keys(iterative_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    x_iterative = iterative_solution_archive[i]; # get the last solution (converged solution)

    # compute the residual for the iterative solution
    r = b - A*x_iterative; # if r ≈ 0, then x_iterative is a solution to the system of equations Ax = b
    norm_r = norm(r); # p = 2 norm of the residual vector

    # check: our solution vs Julia solution, should be approximately equal
    @assert isapprox(norm_r, 0, atol=atol)
end

AssertionError: AssertionError: isapprox(norm_r, 0, atol = atol)

So our iterative method generated a valid solution (if no warnings or assertion errors were raised), but is it approximately equal to the true solution `x_true` that we generated earlier? And, how do the choices of absolute tolerance `atol`, the maximum number of iterations and the choice of method affect the validity of the solution obtained from the iterative method?

Let's check that next!

> __Experiment:__ Compare the norm of the residual vector `r` between the true and iterative solutions with zero for different values of the absolute tolerance `atol`, maximum number of iterations, and different iterative methods (Jacobi, Gauss-Seidel, and SOR). How do these choices influence the validity of the solution obtained from the iterative method?

What do we get?

In [11]:
let

    # initialize -
    atol = 1e-8; # absolute tolerance for checking the residual norm against zero
    i = keys(iterative_solution_archive) |> collect |> sort |> last; # get the last key (the last iteration)
    x_iterative = iterative_solution_archive[i]; # get the last solution (converged solution)

    # compute the norm of the difference between the computed solution and the manufactured solution
    r = x_iterative - x_true; # if r ≈ 0, then x_iterative is approximately equal to x_true
    norm_r = norm(r); # p = 2 norm of the residual vector
    
    # run the test -
    @assert isapprox(norm_r, 0.0; atol = atol); # test
end

AssertionError: AssertionError: isapprox(norm_r, 0.0; atol = atol)

In [12]:
iterative_solution_archive[0]

100-element Vector{Float64}:
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 ⋮
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1

__Interesting!__ It appears that the choice of absolute tolerance `atol` (and the maximum number of iterations) can significantly affect the quality of the solution obtained from the iterative method.

> __Observation (atol):__ For small absolute tolerances (e.g., `atol = 1e-10`), the iterative solution is very close to the true solution, while for larger absolute tolerances (e.g., `atol = 1e-6`), the iterative solution may deviate more from the true solution. This indicates that the choice of `atol` plays a crucial role in determining the accuracy of the iterative solution. 

> __Observation (max iterations):__ The maximum number of iterations allowed for the iterative method also influences the quality of the solution. For small `atol` parameter values, a larger number of iterations may be required to achieve convergence, while for larger `atol` values, fewer iterations may be sufficient to reach a solution that is considered valid.

In summary, both the absolute tolerance `atol` and the maximum number of iterations are important factors that affect whether the iterative solution is considered valid for the system of equations. A smaller `atol` may lead to a more accurate solution, but it may also require more iterations to converge, while a larger `atol` may lead to a quicker solution but with less accuracy.

### Curious: We have the true solution, we can compute the theoretical convergence bound on steps.
Since we have the true solution `x_true`, we can compute the theoretical convergence bound on the number of iterations required for the iterative methods to converge to a solution that is approximately equal to the true solution.

To reach a desired accuracy $\epsilon$ at iteration $k$, we can bound the error as:
$$
\|\mathbf{e}^{(k)}\| \leq q^{k}\,\|\mathbf{e}^{(0)}\| \leq \epsilon
$$
which implies:
$$
\boxed{
   k\geq\frac{\ln(\epsilon/\|\mathbf{e}^{(0)}\|)}{|\ln(\lVert\mathbf{G}\rVert)|}
\quad\blacksquare}
$$

where $\mathbf{G}$ is the iteration matrix for the chosen iterative method, $\mathbf{e}^{(0)}$ is the initial error (i.e., the difference between the initial guess and the true solution), and $q = \lVert\mathbf{G}\rVert$ is some norm of the iteration matrix.

In [13]:
let

    # initialize -
    ϵ = 1e-8; # convergence tolerance
    xₒ = iterative_solution_archive[0]; # initial guess
    eₒ = x_true - xₒ; # initial error
    D = diag(A) |> a-> diagm(a); # extract the diagonal of A and form a diagonal matrix
    U = triu(A,1); # extract the upper triangular part of A, excluding the diagonal
    L = tril(A,-1); # extract the lower triangular part of A, excluding the diagonal

    # TODO: The iteration matrix will change for each method, so you will need to implement this for each method
    # TODO: Uncomment for Jacobi method
    # M = D;
    # N = -(L + U);
    # G = inv(M) * N; # iteration matrix

    # TODO: Uncomment for Gauss-Seidel method
    M = D + L;
    N = -U;
    G = inv(M) * N; # iteration matrix

    # compute k -
    k = abs(log(ϵ/norm(eₒ, Inf))) / abs(log(norm(G, Inf)));
end

26.42322779170013

### Curious: Performance of iterative versus QR decomposition methods for solving linear equations?
So I'm curious about how the performance of our iterative methods compares to the QR decomposition method for solving linear equations. Let's set up a benchmarking experiment (using [the `@benchmark` macro exported from the `BenchmarkTools.jl` package](https://github.com/JuliaCI/BenchmarkTools.jl)) to compare the time taken by each method to solve the same system of equations.

Let's start with the QR decomposition method and then compare it to the iterative methods.

In [ ]:
@benchmark begin
    Q,R = qr($A);
    x_qr = R \ (transpose(Q) * $b);
end

Next, let's look at the performance of our iterative methods. Use the same setup as above to get an apples to apples comparison of the time and memory taken our iterative method versus the QR decomposition method.

In [ ]:
let
    
    # initialize -
    number_of_rows = size(A, 1);
    maximum_number_of_iterations = 25000; # maximum number of iterations for the iterative method
    tolerance = 1e-12;
    algorithm = GaussSeidelMethod(); # change this to Jacobi(), GaussSeidelMethod() or SuccessiveOverRelaxationMethod() to test other algorithms
    ω = 0.6; # relaxation factor, only used for SOR
    xₒ = 0.1*ones(number_of_rows); # initial solution guess xₒ

     # call the solve method with the appropriate parameters -
    @benchmark VLDataScienceMachineLearningPackage.solve($A, $b, $xₒ; algorithm = $algorithm, 
        maxiterations = $maximum_number_of_iterations, 
        ϵ = $tolerance, 
        ω = $ω # only used for SOR
    );
end

__Wow!__ We are slow (and use a lot of memory) compared to the QR decomposition method for solving linear equations. Hmmm. Maybe we should consider doing some refactoring! But that is a task for another day. For now, let's just note that the QR decomposition method is significantly faster and more memory efficient than our iterative methods for solving linear equations in this case.
___

## Summary
In this lab, we compared direct and iterative methods for solving linear systems using a manufactured solution approach for the 1D Poisson equation. We implemented QR decomposition and iterative solvers to understand their trade-offs in accuracy, convergence, and performance.

> **Key takeaways:**
> * **Direct methods outperform iterative methods for dense systems.** QR decomposition achieved machine precision accuracy and significantly faster execution times compared to Jacobi, Gauss-Seidel, and SOR methods on our tridiagonal test problem.
> * **Manufactured solutions enable rigorous validation.** By constructing test problems with known exact solutions, we quantitatively measured solver accuracy and verified that our numerical methods converge to the correct answer.
> * **Convergence theory predicts practical performance.** The diagonal dominance properties we checked theoretically guaranteed convergence of our iterative methods, demonstrating the connection between mathematical theory and computational results.

Here we explored square systems of linear equations. In practice, many real-world problems are not square. Thus, in future labs we will explore least squares solutions and regularization techniques for nonsquare systems.

___